In [1]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import astropy
import lightkurve as lk
from scipy import *
from astropy import constants as ac
from astropy import units as u

In [2]:
from matplotlib import rcParams
rcParams["savefig.dpi"] = 100
rcParams["figure.dpi"] = 100
rcParams["font.size"] = 20
rcParams["figure.figsize"] = [8, 5]
rcParams["font.family"] = "sans-serif"
rcParams["font.sans-serif"] = ["Computer Modern Sans Serif"]
rcParams["text.usetex"] = True
plt.rcParams['axes.linewidth']=1.5

Loading in lists of targets

In [3]:
candidates = pd.read_csv('/Users/lupitatovar/Cool-Worlds/targets/candidates_temp_cuts.csv')
confirmed = pd.read_csv('/Users/lupitatovar/Cool-Worlds/targets/confirmed_temp_cuts.csv')

In [4]:
stars = pd.read_csv('/Users/lupitatovar/Cool-Worlds/targets/stellar_propoerties.csv')

In [5]:
conf_stars = np.where(stars["Status"].str.contains("Confirmed"))[0]
cand_stars = np.where(stars["Status"].str.contains("Candidate"))[0]

In [6]:
len(stars['st_radius'].values[cand_stars])

35

Defining a function to calculate TSM based on (Kempton et al. 2018) paper

In [7]:
def tsm(sf,Rp,Teq,Mp,Rs,mj):
    #sf = 0.190 (for planets with radii < 1.5 earth radius)
    #sf = 1.26 for planets 1.5<rp<2.75
    tsm_ = sf * ((Rp**3 * Teq) / (Mp * Rs**2)) * 10**(-mj/5)
    return tsm_

Calculating TSM for all the confirmed planets

In [8]:
rp_conf = confirmed['rade'].values             #[Earth radii]
mp_conf = confirmed['masse_est'].values              #[Earth mass, estimate using m/r relationship]
temp_conf = confirmed['T_eff'].values             #effective temperature
rs_conf =  stars['st_radius'].values[conf_stars]   #stellar radii
jmag_conf = stars['Jmag'].values[conf_stars]       #magnitude in j band

In [9]:
print(rp_conf)

[1.72227027 1.8        1.533375   1.6        1.56843636 1.55647059]


In [10]:
tsm_conf = []

for i in range (0,len(confirmed)):
    tsm_temp = tsm(1.26,rp_conf[i],temp_conf[i],mp_conf[i],rs_conf[i],jmag_conf[i])
    tsm_conf = np.append(tsm_conf,tsm_temp)
    print (tsm_conf)

[2.7586628]
[2.7586628  3.65611185]
[2.7586628  3.65611185 2.50550157]
[2.7586628  3.65611185 2.50550157 5.80063999]
[2.7586628  3.65611185 2.50550157 5.80063999 2.6136068 ]
[2.7586628  3.65611185 2.50550157 5.80063999 2.6136068  2.5946604 ]


In [11]:
tsm_conf

array([2.7586628 , 3.65611185, 2.50550157, 5.80063999, 2.6136068 ,
       2.5946604 ])

In [12]:
confirmed

,Unnamed: 0,name,hostname,insol,period,T_eff,rade,semi_au,masse_est,eccen,st_log_lum,discoverymethod,H_Tidal,H_Radio,H_Total
0,2,Kepler-1544 b,Kepler-1544,0.774587,168.811174,248.740874,1.722270,0.539467,3.613241,0.00,-0.647,Kepler,0.000000e+00,2.966220e+14,2.966220e+14
1,4,Kepler-296 f,Kepler-296,0.620000,63.336270,212.794092,1.800000,0.255000,3.894446,0.33,-1.569,Kepler,2.045600e+13,3.386230e+14,3.590790e+14
2,6,Kepler-62 f,Kepler-62,0.509360,267.291000,223.993704,1.533375,0.722749,2.966459,0.00,-0.575,Kepler,0.000000e+00,2.093360e+14,2.093360e+14
3,8,Kepler-1652 b,Kepler-1652,0.810000,38.097220,253.928767,1.600000,0.165400,3.188594,0.00,-1.638,Kepler,0.000000e+00,2.378260e+14,2.378260e+14
4,9,Kepler-441 b,Kepler-441,0.241811,207.248200,185.929645,1.568436,0.558807,3.082536,0.10,-1.122,Kepler,2.515201e+09,2.240270e+14,2.240290e+14
5,10,Kepler-1229 b,Kepler-1229,0.629999,86.828989,236.218911,1.556471,0.315740,3.042715,0.00,-1.202,Kepler,0.000000e+00,2.189380e+14,2.189380e+14


In [13]:
confirmed.insert(15,'TSM',tsm_conf)

In [14]:
confirmed

,Unnamed: 0,name,hostname,insol,period,T_eff,rade,semi_au,masse_est,eccen,st_log_lum,discoverymethod,H_Tidal,H_Radio,H_Total,TSM
0,2,Kepler-1544 b,Kepler-1544,0.774587,168.811174,248.740874,1.722270,0.539467,3.613241,0.00,-0.647,Kepler,0.000000e+00,2.966220e+14,2.966220e+14,2.758663
1,4,Kepler-296 f,Kepler-296,0.620000,63.336270,212.794092,1.800000,0.255000,3.894446,0.33,-1.569,Kepler,2.045600e+13,3.386230e+14,3.590790e+14,3.656112
2,6,Kepler-62 f,Kepler-62,0.509360,267.291000,223.993704,1.533375,0.722749,2.966459,0.00,-0.575,Kepler,0.000000e+00,2.093360e+14,2.093360e+14,2.505502
3,8,Kepler-1652 b,Kepler-1652,0.810000,38.097220,253.928767,1.600000,0.165400,3.188594,0.00,-1.638,Kepler,0.000000e+00,2.378260e+14,2.378260e+14,5.800640
4,9,Kepler-441 b,Kepler-441,0.241811,207.248200,185.929645,1.568436,0.558807,3.082536,0.10,-1.122,Kepler,2.515201e+09,2.240270e+14,2.240290e+14,2.613607
5,10,Kepler-1229 b,Kepler-1229,0.629999,86.828989,236.218911,1.556471,0.315740,3.042715,0.00,-1.202,Kepler,0.000000e+00,2.189380e+14,2.189380e+14,2.594660


In [15]:
#confirmed.to_csv('/Users/lupitatovar/Cool-Worlds/targets/confirmed_tsm.csv')

Calculating TSM for all of the candidates

In [16]:
rp_cand = candidates['Radius'].values             #[Earth radii]
mp_cand = candidates['masse_est'].values              #[Earth mass, estimate using m/r relationship]
temp_cand = candidates['T_eff'].values             #effective temperature
rs_cand =  stars['st_radius'].values[cand_stars]   #stellar radii
jmag_cand = stars['Jmag'].values[cand_stars]       #magnitude in j band

In [17]:
tsm_cand = []

for i in range (0,len(candidates)):
    tsm_temp2 = tsm(1.26,rp_cand[i],temp_cand[i],mp_cand[i],rs_cand[i],jmag_cand[i])
    tsm_cand = np.append(tsm_cand,tsm_temp2)
    print (tsm_temp2)

2.6276495343022037
1.4013438943977512
0.839123963396286
1.533627994338674
1.5207631245030278
1.8836854795644469
1.1341439780554852
1.6593738896313124
1.6552748789954765
1.1813498334989512
1.8957017517268229
1.2539558606294596
2.402928896286327
2.215981740947318
1.0233562870313442
0.796798268209471
1.5068859044417853
2.2534284151509483
1.1872818252639026
1.2944378225900737
1.5184480314928381
2.4275167857984097
1.1726694983915869
3.8505970259189857
1.4984709146114947
1.6262722823327929
1.125112555686863
1.183850281430631
2.7487811903045585
1.3906284079466082
0.7331447846443261
25.6208823517275
35.15844003997162
32.477348023133246
23.33617206013681


In [18]:
tsm_cand

array([ 2.62764953,  1.40134389,  0.83912396,  1.53362799,  1.52076312,
        1.88368548,  1.13414398,  1.65937389,  1.65527488,  1.18134983,
        1.89570175,  1.25395586,  2.4029289 ,  2.21598174,  1.02335629,
        0.79679827,  1.5068859 ,  2.25342842,  1.18728183,  1.29443782,
        1.51844803,  2.42751679,  1.1726695 ,  3.85059703,  1.49847091,
        1.62627228,  1.12511256,  1.18385028,  2.74878119,  1.39062841,
        0.73314478, 25.62088235, 35.15844004, 32.47734802, 23.33617206])

In [19]:
candidates.insert(14,'TSM',tsm_cand)

In [20]:
candidates

,Unnamed: 0,name,hostname,Insolation,Period,T_eff,Radius,masse_est,semi_au,st_log_lum,Discovery,H_Tidal,H_Radio,H_Total,TSM
0,0,KOI-4427.01,KIC 4172805,0.318849,147.661729,199.239771,1.797673,3.885904,0.447412,-1.195000,Kepler,4.269050e+10,3.373120e+14,3.373540e+14,2.627650
1,1,KOI-4848.01,KIC 9005321,0.160519,384.696100,167.826873,1.880000,4.192848,0.903101,-0.883000,Kepler,2.197108e+10,3.858090e+14,3.858310e+14,1.401344
2,2,KOI-7716.01,KIC 5097856,0.434274,483.364050,215.238882,1.503482,2.868943,1.153766,-0.238000,Kepler,4.704759e+08,1.973300e+14,1.973310e+14,0.839124
3,4,KOI-5087.01,KIC 4770798,0.332875,651.074310,201.395598,1.141911,1.563745,1.441648,-0.160000,Kepler,1.584473e+07,8.645620e+13,8.645620e+13,1.533628
4,5,KOI-5227.01,KIC 6034834,0.598982,371.564920,233.256197,1.611944,3.229113,0.967816,-0.251000,Kepler,2.518710e+09,2.431920e+14,2.431940e+14,1.520763
5,6,KOI-5236.01,KIC 6067545,0.811569,550.865140,251.658130,1.921435,4.350944,1.334558,0.160000,Kepler,6.994064e+08,4.118850e+14,4.118860e+14,1.883685
6,7,KOI-5130.01,KIC 5166811,0.430720,370.059180,214.797110,1.900000,4.268858,0.938774,-0.420683,Kepler,1.127595e+09,3.982540e+14,3.982550e+14,1.134144
7,8,KOI-5408.01,KIC 7631138,0.354582,415.748785,204.601495,1.710821,3.572556,1.034289,-0.421000,Kepler,3.826753e+08,2.907450e+14,2.907460e+14,1.659374
8,9,KOI-5789.01,KIC 10387792,0.319386,383.262170,199.323573,1.532865,2.964783,0.944805,-0.545000,Kepler,5.560228e+08,2.091270e+14,2.091280e+14,1.655275
9,10,KOI-5798.01,KIC 10487228,0.403587,318.258580,211.331357,1.963136,4.512475,0.834702,-0.551000,Kepler,2.157382e+09,4.392890e+14,4.392910e+14,1.181350


In [21]:
#candidates.to_csv('/Users/lupitatovar/Cool-Worlds/targets/cadidates_tsm.csv')